In [ ]:
# !nvidia-smi

In [3]:
import torch
print(torch.backends.mps.is_available())

True


In [1]:
# !pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 699.7/699.7 kB 7.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 10.4 MB/s eta 0:00:0000:0100:01


# Install YOLO

In [5]:
from ultralytics import YOLO
import os
from IPython.display import display, Image
from IPython import display
display.clear_output()

# Train YOLOv8 Model on Custom Dataset

In [6]:
# !pip install roboflow

In [7]:
from roboflow import Roboflow
rf = Roboflow(api_key="BjsTUwG8DMvVTG2rcnyz")
project = rf.workspace("mario-28fvy").project("mario_detection_simple")
dataset = project.version(1).download("yolov8")

loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.1.2, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to Mario_detection_simple-1 in yolov8:: 100%|██████████| 178/178 [00:00<00:00, 6782.77it/s]


In [11]:
!yolo task=detect mode=train model=yolov8m.pt data={dataset.location}/data.yaml epochs=20 imgsz=640 device=mps

Ultralytics YOLOv8.1.2 🚀 Python-3.11.5 torch-2.1.2 MPS (Apple M3 Pro)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/Users/maximvandecasteele/PycharmProjects/master2/mario_phase1/Mario_detection_simple-1/data.yaml, epochs=20, time=None, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=mps, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, sav

# Prediction (playground)

In [72]:
# run this if you want to see bounding boxes on images
# !yolo task=detect mode=predict model=/Users/maximvandecasteele/PycharmProjects/master2/mario_phase0/models/Mario_OD_vanilla_best.pt conf=0.5 source=/Users/maximvandecasteele/PycharmProjects/master2/mario_phase0/demo/

Ultralytics YOLOv8.1.2 🚀 Python-3.11.5 torch-2.1.2 CPU (Apple M3 Pro)
Model summary (fused): 218 layers, 25844392 parameters, 0 gradients, 78.7 GFLOPs

image 1/2 /Users/maximvandecasteele/PycharmProjects/master2/mario_phase0/demo/img_SuperMarioBros-1-1-v0_2024-01-17-16_06_44.png: 608x640 1 goomba, 182.8ms
image 2/2 /Users/maximvandecasteele/PycharmProjects/master2/mario_phase0/demo/img_SuperMarioBros-1-1-v0_2024-01-17-16_08_35.png: 608x640 4 brickss, 2 goombas, 2 surprise_boxs, 173.2ms
Speed: 1.2ms preprocess, 178.0ms inference, 0.5ms postprocess per image at shape (1, 3, 608, 640)
Results saved to runs/detect/predict
💡 Learn more at https://docs.ultralytics.com/modes/predict


In [67]:
import pandas as pd

# return bounding boxes
model = YOLO('/Users/maximvandecasteele/PycharmProjects/master2/mario_phase0/models/Mario_OD_vanilla_best.pt')


# just load an image or list of images into the model for prediction
# for result object   https://docs.ultralytics.com/modes/predict/#working-with-results
results = model('/Users/maximvandecasteele/PycharmProjects/master2/mario_phase0/demo/img_SuperMarioBros-1-1-v0_2024-01-17-16_08_35.png')

boxes = None
for r in results:
    boxes = r.boxes.numpy()
    classes = pd.DataFrame(boxes.cls, columns=['class'])
    xywh = pd.DataFrame(boxes.xywh, columns=['x', 'y', 'w', 'h'])
    positions = pd.concat([classes,xywh], axis=1)

positions = pd.DataFrame(data=None, columns=['name', 'xmin', 'xmax', 'ymin', 'ymax'])

for r in results:
    #     TODO convert classes into names
    boxes = r.boxes.numpy()
    classes = pd.DataFrame(boxes.cls, columns=['class'])
    # other types of bounding box data can be chosen: xyxy, xywh, xyxyn, xywhn
    xywh = pd.DataFrame(boxes.xywh, columns=['x', 'y', 'w', 'h'])
    positions = pd.concat([classes, xywh], axis=1)


image 1/1 /Users/maximvandecasteele/PycharmProjects/master2/mario_phase0/img_SuperMarioBros-1-1-v0_2024-01-17-16_06_44.png: 608x640 1 goomba, 1 surprise_box, 213.2ms
Speed: 1.6ms preprocess, 213.2ms inference, 0.7ms postprocess per image at shape (1, 3, 608, 640)


In [68]:
positions

,class,x,y,w,h
0,1.0,48.643257,200.134109,13.549282,17.219238
1,6.0,39.913296,190.653275,79.343910,36.293121
